In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
train_data = pd.read_csv("../dataset/preprocessed/netflix_train.csv")
val_data = pd.read_csv("../dataset/preprocessed/netflix_val.csv")
test_data = pd.read_csv("../dataset/preprocessed/netflix_test.csv")

X_train = train_data[['title', 'description']]
y_train = train_data['target_ages']

X_val = val_data[['title', 'description']]
y_val = val_data['target_ages']

X_test = test_data[['title', 'description']]
y_test = test_data['target_ages']


label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)

vectorizer = CountVectorizer()
X_train_title = vectorizer.fit_transform(X_train['title'])
X_val_title = vectorizer.transform(X_val['title'])
X_test_title = vectorizer.transform(X_test['title'])

X_train_desc = vectorizer.fit_transform(X_train['description'])
X_val_desc = vectorizer.transform(X_val['description'])
X_test_desc = vectorizer.transform(X_test['description'])

X_train = pd.concat([pd.DataFrame(X_train_title.toarray()), pd.DataFrame(X_train_desc.toarray())], axis=1)
X_val = pd.concat([pd.DataFrame(X_val_title.toarray()), pd.DataFrame(X_val_desc.toarray())], axis=1)
X_test = pd.concat([pd.DataFrame(X_test_title.toarray()), pd.DataFrame(X_test_desc.toarray())], axis=1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)


MemoryError: Unable to allocate 921. MiB for an array with shape (6155, 19604) and data type float64

In [ ]:
# MLP
model = Sequential([
    Dense(128, activation='relu', input_dim=X_train.shape[1]),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))



Epoch 1/10


193/193 [==============================] - 7s 35ms/step - loss: 1.4197 - accuracy: 0.4450 - val_loss: 1.1291 - val_accuracy: 0.4928
Epoch 2/10
193/193 [==============================] - 7s 34ms/step - loss: 0.2719 - accuracy: 0.9173 - val_loss: 1.3615 - val_accuracy: 0.5019
Epoch 3/10
193/193 [==============================] - 6s 31ms/step - loss: 0.0174 - accuracy: 0.9966 - val_loss: 1.5045 - val_accuracy: 0.4989
Epoch 4/10
193/193 [==============================] - 6s 33ms/step - loss: 0.0029 - accuracy: 0.9997 - val_loss: 1.5997 - val_accuracy: 0.4989
Epoch 5/10
193/193 [==============================] - 6s 32ms/step - loss: 0.0014 - accuracy: 0.9998 - val_loss: 1.6787 - val_accuracy: 0.4996
Epoch 6/10
193/193 [==============================] - 6s 32ms/step - loss: 0.0018 - accuracy: 0.9997 - val_loss: 1.7126 - val_accuracy: 0.4981
Epoch 7/10
193/193 [==============================] - 7s 36ms/step - loss: 5.7574e-04 - accuracy: 0.9998 - val_loss: 1.7366 - val_accuracy

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")

y_pred_probs = model.predict(X_test)
y_pred = y_pred_probs.argmax(axis=1)

y_test_original = label_encoder.inverse_transform(y_test)
y_pred_original = label_encoder.inverse_transform(y_pred)

print(classification_report(y_test_original, y_pred_original))

42/42 [==============================] - 0s 3ms/step - loss: 1.8211 - accuracy: 0.5227
Test Accuracy: 0.5227272510528564
42/42 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

      Adults       0.60      0.66      0.63       614
        Kids       0.57      0.44      0.50        66
  Older Kids       0.51      0.42      0.46       242
       Teens       0.39      0.38      0.39       398

    accuracy                           0.52      1320
   macro avg       0.52      0.48      0.49      1320
weighted avg       0.52      0.52      0.52      1320

